# Codes for (Question1-5,9)
### Analyzing a Mental Health Dataset


#### Team members: ***Frank Xu, Allison Young, Abhishek Angadi, Alan Zhuangdie Zhou***

In [152]:
# Please Install package findspark and pyspark from your terminal if you have not
#import findspark

# Please replace the directory to your own one for spark
#findspark.init('/Users/frank/workspace/spark-2.4.0-bin-hadoop2.7/')
import pyspark

# Creating spark context
conf = pyspark.SparkConf().setMaster("local").setAppName("Spark Project")
sc = pyspark.SparkContext(conf = conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Python Spark SQL basic example, master=local[*]) created by getOrCreate at <ipython-input-2-499f49df3242>:4 

In [153]:
# Reading in all the data
bg = sc.textFile("VDL2011_Background.txt")
md = sc.textFile("VDL2011_Meds.txt")
pd = sc.textFile("VDL2011_PDiagnose.txt")
cgi = sc.textFile("VDL2011_CGI.txt")
ps = sc.textFile("VDL2011_Patient_Service.txt")
tp = sc.textFile("VDL2011_TypePatient.txt")

# Please only run this chunk one time
# If it returns a Value Error saying cannot run multiple SparkContexts at once, 
# just run the last chunk of this page: sc.stop() to stop this sc,
# Then go back to this chunk

NameError: name 'sc' is not defined

In [3]:
# Question 2
# Basic descriptive statistics: How many hospitals are represented in the data? 
# What is the average number of patients per hospital? Minimum and maximum?

# Mapping
def q2map(s):
    temp = s.split(',')
    return temp[0], 1

uniquehosp = bg.map(q2map).reduceByKey(lambda a,b: a+b).sortBy(lambda x:x[1]).collect()

# Calculating and printing the average patients per hospital
patientcount = bg.count()
hospitalcount = len(uniquehosp) -1
print('Number of hospitals represented in the data is:',hospitalcount)
print('Average patient per hospital: {}'.format(patientcount/hospitalcount))
print('The hospital with minimum patients is {}, with {} patients.'.format(uniquehosp[1][0], uniquehosp[1][1]))
print('The hospital with maximum patients is {}, with {} patients.'.format(uniquehosp[-1][0], uniquehosp[-1][1]))


Number of hospitals represented in the data is: 17
Average patient per hospital: 10844.823529411764
The hospital with minimum patients is eleanor2, with 240 patients.
The hospital with maximum patients is Duke, with 65443 patients.


In [77]:
# Question 3
# Our study has decided to focus on depression and depression-related conditions
# (Bipolar Disorder, Dysthymic Disorder, etc.).  
# How many of the patients have a depression or depression related diagnosis?

# Mapper function
def q3map(s):
    temp = s.split(',')
    return temp[0]+temp[1],temp[4]

# Running map-filter-reducer
depressed = pd.map(q3map).filter(lambda x:'296'in x[1].split('.')[0] or '300'in x[1].split('.')[0]or '301.1'in x[1] or '311'in x[1].split('.')[0]).reduceByKey(lambda a,b:a+b).collect()
# What string do we want?
#Sum up all the depression or depression related diagnosis
print('The number of depression or depression related diagnosis is:',len(depressed)-1)

The number of depression or depression related diagnosis is: 90814


In [5]:
# Question 4
# How many unique Psych drugs are in the data (check the “PsyMed” column)?
import string
# Mapper
def q4map(s):
    temp = s.split(',')
    return temp[5]+' |'+temp[25],1
import string
# Run map-reducer
psymed = md.map(q4map).filter(lambda x: 'Yes' in x[0]).reduceByKey(lambda a,b:a+b).sortBy(lambda x:x[0]).collect()
# Find the unique Psych drugs numbers
uniquepsych = set([x[0].split(" ")[0].lower().strip(string.punctuation) for x in psymed])
uniquepsych2 = set([x.split("-")[0] for x in uniquepsych])
print('There are {} unique psych drugs in the data.'.format(len(uniquepsych2)-1))

There are 416 unique psych drugs in the data.


In [82]:
len(psymed)

2672

In [ ]:
# Question 5 
# Let’s start getting some useful results. 
# What are the most common psych meds for patients with Major Depressive Disorder? 
# For any diagnosis related to depression? What about Cyclothymia?


In [120]:
def q5map1(s):
    temp = s.split(',')
    return temp[0]+'|'+temp[1]+'|'+temp[2],temp[4]

pdset = pd.map(q5map1).filter(lambda x:'296'in x[1].split('.')[0] and 
    int(x[1].split('|')[0].split('.')[1]) in range(20,37)).reduceByKey(lambda a,b:a+b)

In [28]:
def q5map2(s):
    temp = s.split(',')
    return temp[0]+'|'+temp[1]+'|'+temp[2],temp[25]+'|'+temp[5].lower()

mdset = md.map(q5map2).filter(lambda x: 'Yes' in x[1])

In [29]:
pdmdjoined = pdset.join(mdset)

In [117]:
pdmdjoined.take(10)

[('ArapahoeHouse|225|143580', ('296.33', 'Yes|prozac-solution')),
 ('ArapahoeHouse|225|143580', ('296.33', 'Yes|neurontin')),
 ('ArapahoeHouse|225|224900', ('296.33', 'Yes|prozac')),
 ('ArapahoeHouse|225|224900', ('296.33', 'Yes|neurontin')),
 ('ArapahoeHouse|3001|318497', ('296.22', 'Yes|clorazepate')),
 ('ArapahoeHouse|3001|318497', ('296.22', 'Yes|prozac')),
 ('ArapahoeHouse|3771|148481', ('296.32', 'Yes|amitriptyline')),
 ('ArapahoeHouse|3771|148481', ('296.32', 'Yes|bupropion')),
 ('ArapahoeHouse|3771|148481', ('296.32', 'Yes|levothyroxine sodium')),
 ('ArapahoeHouse|4460|173084', ('296.34', 'Yes|fluoxetine hcl'))]

In [32]:
def q5map3(s):
    for i in range(0,len(s)):
        return s[1][1],1

drugs = pdmdjoined.map(q5map3).reduceByKey(lambda a,b:a+b)

In [33]:
drugs.take(10)

[('Yes|paxil 20mg titr', 62),
 ('Yes|vistaril', 9608),
 ('Yes|haldol', 8642),
 ('Yes|abilify solution', 102),
 ('Yes|venlafaxine', 1262),
 ('Yes|ativan-inj', 279),
 ('Yes|duragesic', 474),
 ('Yes|risperdal 1mg hs', 21),
 ('Yes|atarax 30-50mg hs prn', 6),
 ('Yes|acetaminophen; codeine', 11)]

In [37]:
def q5map4(s):
    for i in range(0,len(s)):
        return s[0].split('|')[1].split(' ')[0],s[1]
    
drugs1 = drugs.map(q5map4).reduceByKey(lambda a,b:a+b)

In [42]:
drugs1.take(10)

[('mirtazapine', 6070),
 ('buspar', 13743),
 ('benadryl-syrup', 9),
 ('methylphenidate-sa', 21),
 ('carbamazepine', 416),
 ('diphenhydramine-inj', 7),
 ('marplan', 60),
 ('cylert', 2),
 ('parlodel', 72),
 ('elavil', 3772)]

In [46]:
def q5map5(s):
    for i in range(0,len(s)):
        return s[0].split('-')[0],s[1]
    
drugs2 = drugs1.map(q5map5).reduceByKey(lambda a,b:a+b).sortBy(lambda x:x[1],ascending=False).collect()

In [51]:
drugs2[0:10]

[('seroquel', 78860),
 ('ambien', 76890),
 ('trazodone', 73810),
 ('lexapro', 73139),
 ('klonopin', 72800),
 ('zoloft', 72048),
 ('abilify', 64320),
 ('wellbutrin', 61768),
 ('prozac', 58787),
 ('effexor', 51494)]

In [48]:
print('The most common psych meds for patients with Major Depressive Disorder is:',drugs2[0][0])

The most common psych meds for patients with Major Depressive Disorder is: seroquel


In [66]:
def q5map11(s):
    temp = s.split(',')
    return temp[0]+'|'+temp[1]+'|'+temp[2],temp[4]

pdset2 = pd.map(q5map11).filter(lambda x:'296'in x[1].split('.')[0] or '300'in x[1].split('.')[0]or '301.1'in x[1] or '311'in x[1].split('.')[0]).reduceByKey(lambda a,b:a+b)

In [69]:
pdmdjoined2 = pdset2.join(mdset)

In [70]:
drugs11 = pdmdjoined2.map(q5map3).reduceByKey(lambda a,b:a+b)

In [72]:
drugs11.take(10)

[('Yes|haldol', 28080),
 ('Yes|vistaril', 21516),
 ('Yes|paxil 20mg titr', 192),
 ('Yes|duragesic', 739),
 ('Yes|ativan-inj', 715),
 ('Yes|abilify solution', 480),
 ('Yes|venlafaxine', 2311),
 ('Yes|risperdal 1mg hs', 21),
 ('Yes|acetaminophen; codeine', 54),
 ('Yes|atarax 30-50mg hs prn', 6)]

In [73]:
drugs22 = drugs11.map(q5map4).reduceByKey(lambda a,b:a+b)

In [74]:
drugs22.take(10)

[('buspar', 31156),
 ('mirtazapine', 8458),
 ('carbamazepine', 2773),
 ('benadryl-syrup', 111),
 ('methylphenidate-sa', 75),
 ('innopran', 6),
 ('diphenhydramine-inj', 12),
 ('elavil', 8079),
 ('marplan', 63),
 ('cylert', 180)]

In [75]:
drugs33 = drugs22.map(q5map5).reduceByKey(lambda a,b:a+b).sortBy(lambda x:x[1],ascending=False).collect()

In [77]:
drugs33[0:10]

[('seroquel', 228070),
 ('abilify', 174841),
 ('klonopin', 165965),
 ('ambien', 149375),
 ('trazodone', 149242),
 ('zoloft', 135880),
 ('lexapro', 135185),
 ('depakote', 121058),
 ('risperdal', 107547),
 ('prozac', 107355)]

In [78]:
print('The most common psych meds for patients with any diagnosis related to depression is:',drugs33[0][0])

The most common psych meds for patients with any diagnosis related to depression is: seroquel


In [79]:
def q5map111(s):
    temp = s.split(',')
    return temp[0]+'|'+temp[1]+'|'+temp[2],temp[4]

pdset3 = pd.map(q5map111).filter(lambda x:'301.1'in x[1]).reduceByKey(lambda a,b:a+b)

In [80]:
pdset3.take(10)

[('Columbia|2553|30898', '301.13'),
 ('Columbia|3328|36946', '301.13'),
 ('Columbia|3502|35498', '301.13'),
 ('Columbia|3854|9218', '301.13'),
 ('Columbia|3854|40487', '301.13'),
 ('Columbia|6399|20240', '301.13'),
 ('Duke|2296|182173', '301.13'),
 ('Duke|2296|352929', '301.13'),
 ('Duke|10616|163133', '301.13'),
 ('Duke|17849|332705', '301.13')]

In [81]:
pdmdjoined3 = pdset3.join(mdset)

In [82]:
drugs111 = pdmdjoined3.map(q5map3).reduceByKey(lambda a,b:a+b)

In [83]:
drugs111.take(10)

[('Yes|haldol', 144),
 ('Yes|abilify solution', 27),
 ('Yes|vistaril', 1),
 ('Yes|trileptal', 63),
 ('Yes|methylin er', 8),
 ('Yes|benztropine', 6),
 ('Yes|moban', 1),
 ('Yes|risperdal', 187),
 ('Yes|oxycontin', 14),
 ('Yes|cymbalta', 62)]

In [84]:
drugs222 = drugs111.map(q5map4).reduceByKey(lambda a,b:a+b)

In [85]:
drugs222.take(10)

[('buspar', 251),
 ('carbamazepine', 1),
 ('elavil', 46),
 ('doxepin', 3),
 ('clorazepate', 45),
 ('oxycodone;', 5),
 ('wellbutrin-sr', 22),
 ('anafranil', 26),
 ('sertraline', 8),
 ('strattera', 151)]

In [86]:
drugs333 = drugs222.map(q5map5).reduceByKey(lambda a,b:a+b).sortBy(lambda x:x[1],ascending=False).collect()

In [87]:
drugs333[0:10]

[('abilify', 1268),
 ('concerta', 896),
 ('lamictal', 818),
 ('vyvanse', 739),
 ('risperdal', 683),
 ('prozac', 624),
 ('seroquel', 623),
 ('klonopin', 617),
 ('zoloft', 545),
 ('trazodone', 506)]

In [88]:
print('The most common psych meds for patients with Cyclothymia is:',drugs333[0][0])

The most common psych meds for patients with Cyclothymia is: abilify


In [ ]:
# Question 6 
# Similarly, what are the most common drugs prescribed in patients diagnosed with Bipolar Disorder? 
# Does this vary appreciably by hospital?

In [94]:
def q6map1(s):
    temp = s.split(',')
    return temp[0]+'|'+temp[1]+'|'+temp[2],temp[4]

pd6set1 = pd.map(q6map1).filter(lambda x:('296'in x[1].split('.')[0] and 
    int(x[1].split('|')[0].split('.')[1]) in range(40,67)) or ('296'in x[1].split('.')[0] and 
    int(x[1].split('|')[0].split('.')[1]) in range(7,9))or ('296'in x[1].split('.')[0] and 
    int(x[1].split('|')[0].split('.')[1])== '89')).reduceByKey(lambda a,b:a+b)

In [97]:
pd6set1.take(10)

[('ArapahoeHouse|62|69698', '296.7'),
 ('ArapahoeHouse|62|132304', '296.7'),
 ('ArapahoeHouse|62|281418', '296.7'),
 ('ArapahoeHouse|548|312035', '296.7'),
 ('ArapahoeHouse|672|132586', '296.7'),
 ('ArapahoeHouse|672|235750', '296.7'),
 ('ArapahoeHouse|672|282814', '296.7'),
 ('ArapahoeHouse|1574|4927', '296.54'),
 ('ArapahoeHouse|1574|23427', '296.54'),
 ('ArapahoeHouse|1574|23617', '296.54')]

In [99]:
pdmd6joined1 = pd6set1.join(mdset)

In [101]:
drugs61 = pdmd6joined1.map(q5map3).reduceByKey(lambda a,b:a+b)

In [103]:
drugs61.take(10)

[('Yes|vistaril', 3955),
 ('Yes|haldol', 10884),
 ('Yes|ativan-inj', 271),
 ('Yes|venlafaxine', 126),
 ('Yes|acetaminophen; codeine', 39),
 ('Yes|zyprexa-zydis', 12),
 ('Yes|clomipramine 50mg', 25),
 ('Yes|abilify solution', 9),
 ('Yes|paxil 20mg titr', 9),
 ('Yes|duragesic', 3)]

In [104]:
drugs62 = drugs61.map(q5map4).reduceByKey(lambda a,b:a+b)

In [105]:
drugs62.take(10)

[('buspar', 3532),
 ('mirtazapine', 419),
 ('carbamazepine', 962),
 ('diphenhydramine-inj', 7),
 ('marplan', 1),
 ('cylert', 17),
 ('parlodel', 1),
 ('elavil', 785),
 ('doxepin', 872),
 ('clorazepate', 573)]

In [106]:
drugs63 = drugs62.map(q5map5).reduceByKey(lambda a,b:a+b).sortBy(lambda x:x[1],ascending=False).collect()

In [108]:
drugs63[0:10]

[('seroquel', 60193),
 ('depakote', 51953),
 ('abilify', 41831),
 ('lithium', 37537),
 ('lamictal', 35362),
 ('klonopin', 31495),
 ('risperdal', 27640),
 ('trazodone', 25997),
 ('ambien', 23530),
 ('zyprexa', 20209)]

In [109]:
print('The most common psych meds for patients with Bipolar Disorder is:',drugs63[0][0])

The most common psych meds for patients with Bipolar Disorder is: seroquel


In [145]:
def q6map21(s):
    for i in range(0,len(s)):
        return s[0],s[1].split('|')[1].split(' ')[0]

def q6map22(s):
    for i in range(0,len(s)):
        return s[0],s[1].split('-')[0]

mdset1 = mdset.map(q6map21)
mdset2 = mdset1.map(q6map22)
mdset2.take(10)

[('ArapahoeHouse|3|36581', 'clorazepate'),
 ('ArapahoeHouse|3|57245', 'clorazepate'),
 ('ArapahoeHouse|3|67801', 'clorazepate'),
 ('ArapahoeHouse|3|91441', 'clorazepate'),
 ('ArapahoeHouse|3|93834', 'clorazepate'),
 ('ArapahoeHouse|3|105206', 'clorazepate'),
 ('ArapahoeHouse|3|113172', 'clorazepate'),
 ('ArapahoeHouse|3|154609', 'clorazepate'),
 ('ArapahoeHouse|3|166231', 'clorazepate'),
 ('ArapahoeHouse|3|188298', 'clorazepate')]

In [146]:
pdmd6joined2 = pd6set1.join(mdset2)

In [149]:
pdmd6joined2.take(10)

[('ArapahoeHouse|1574|4927', ('296.54', 'abilify')),
 ('ArapahoeHouse|1574|4927', ('296.54', 'tramadol')),
 ('ArapahoeHouse|2587|31120', ('296.7', 'citalopram')),
 ('ArapahoeHouse|2587|31120', ('296.7', 'clorazepate')),
 ('ArapahoeHouse|2587|31120', ('296.7', 'depakote')),
 ('ArapahoeHouse|2587|31120', ('296.7', 'seroquel')),
 ('ArapahoeHouse|2587|186150', ('296.7', 'citalopram')),
 ('ArapahoeHouse|2587|186150', ('296.7', 'clorazepate')),
 ('ArapahoeHouse|2587|186150', ('296.7', 'depakote')),
 ('ArapahoeHouse|2587|186150', ('296.7', 'seroquel'))]

In [182]:
def q6map22(s):
    for i in range(0,len(s)):
        return s[0].split('|')[0]+'|'+s[1][1],1

hosdrug = pdmd6joined2.map(q6map22).reduceByKey(lambda a,b:a+b).sortBy(lambda x:x[0].split('|')[0]).collect()

In [227]:
hos0 = hosdrug[0][0].split('|')[0]
nummax = hosdrug[0][1]
maxdrug = hosdrug[0][0].split('|')[1]
lst = []
for i in range(0,len(hosdrug)):
    hos = hosdrug[i][0].split('|')[0]
    num = hosdrug[i][1]
    if hos0 == hos and num > nummax:
        nummax = num
        maxdrug = hosdrug[i][0].split('|')[1]
    elif hos0 != hos:
        lst.append((hos0,maxdrug,nummax))
        hos0 = hos
        nummax = hosdrug[i][1]
    else:
        pass

In [228]:
lst

[('ArapahoeHouse', 'abilify', 556),
 ('Boulder County', 'abilify', 24),
 ('CampagnaAcademy', 'vyvanse', 1359),
 ('Columbia', 'lithium', 1591),
 ('Duke', 'depakote', 7065),
 ('DukeCCFH', 'abilify', 120),
 ('GrantBlackford', 'abilify', 2824),
 ('LifeSpring', 'seroquel', 8441),
 ('MONTEPROD', 'depakote', 7249),
 ('PorterStarke', 'seroquel', 9493),
 ('SVTN', 'seroquel', 6085),
 ('UCDenver', 'lithium', 511),
 ('UnivColoradoHospital', 'seroquel', 1581),
 ('WestCentral', 'depakote', 360),
 ('Westchester', 'seroquel', 18688)]

In [231]:
print('The most common psych meds for patients with Bipolar Disorder in each hospital differs as below:')
lst

The most common psych meds for patients with Bipolar Disorder in each hospital differs as below:


[('ArapahoeHouse', 'abilify', 556),
 ('Boulder County', 'abilify', 24),
 ('CampagnaAcademy', 'vyvanse', 1359),
 ('Columbia', 'lithium', 1591),
 ('Duke', 'depakote', 7065),
 ('DukeCCFH', 'abilify', 120),
 ('GrantBlackford', 'abilify', 2824),
 ('LifeSpring', 'seroquel', 8441),
 ('MONTEPROD', 'depakote', 7249),
 ('PorterStarke', 'seroquel', 9493),
 ('SVTN', 'seroquel', 6085),
 ('UCDenver', 'lithium', 511),
 ('UnivColoradoHospital', 'seroquel', 1581),
 ('WestCentral', 'depakote', 360),
 ('Westchester', 'seroquel', 18688)]

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
mdsql = spark.read.csv("VDL2011_Meds.txt",header =True)

In [4]:
mdsql.createOrReplaceTempView("mdsql")

In [3]:
#mdsql = spark.sql("select siteID, BackgroundID, ednum, Medication, Onset from mdsql where PsyMed = 'Yes' AND siteID='ArapahoeHouse' ")
#mdsql.toPandas().to_csv('mycsv1.txt')
mdsql = spark.read.csv("mycsv1.txt",header =True)

In [ ]:
# Question 9:
# You have a good-sized collection of data in front of you. 
# Find some interesting patterns. 
# The Clinical Global Impressions (CGI) Scales are used to quickly indicate severity 
# and improvement (since first treatment) of a patient. What inferences can we make about 
# specific medications given the CGI scores? Formulate some hypotheses and test them using the data.

In [5]:
mdsql.createOrReplaceTempView("mdsql")

In [7]:
cgi = spark.read.csv("VDL2011_CGI.txt",header =True)

In [8]:
cgi.createOrReplaceTempView("cgi")

In [9]:
#cgisql = spark.sql("select * from cgi where cgi.Improvement <> 'None' AND cgi.Severity <> 'None'")
#cgisql.toPandas().to_csv('mycsv2.txt')
cgisql = spark.read.csv("mycsv2.txt",header =True)

In [11]:
mdsql9 = spark.sql("select siteID, BackgroundID, ednum, Medication, Onset from mdsql")
mdrdd = mdsql9.rdd.map(tuple)

In [13]:
from string import punctuation
def q7map1(s):
    for i in range(0,len(s)):
        return s[3].split(' ')[0].lower(),s[0:3]

def q7map2(s):
    for i in range(0,len(s)):
        return s[0].split('-')[0].strip(punctuation),s[1:]

mdrdd1 = mdrdd.map(q7map1)
mdrdd2 = mdrdd1.map(q7map2)
mdrdd2.take(10)

[('clorazepate', (('ArapahoeHouse', '3', '36581'),)),
 ('clorazepate', (('ArapahoeHouse', '3', '57245'),)),
 ('clorazepate', (('ArapahoeHouse', '3', '67801'),)),
 ('clorazepate', (('ArapahoeHouse', '3', '91441'),)),
 ('clorazepate', (('ArapahoeHouse', '3', '93834'),)),
 ('clorazepate', (('ArapahoeHouse', '3', '105206'),)),
 ('clorazepate', (('ArapahoeHouse', '3', '113172'),)),
 ('clorazepate', (('ArapahoeHouse', '3', '154609'),)),
 ('clorazepate', (('ArapahoeHouse', '3', '166231'),)),
 ('clorazepate', (('ArapahoeHouse', '3', '188298'),))]

In [14]:
mdcleaned = mdrdd2.map(lambda x: (x[0], x[1][0][0] , x[1][0][1],x[1][0][2])).toDF(["Medication","siteID","BackgroundID","ednum"])

In [15]:
mdcleaned.createOrReplaceTempView("mdcleaned")

In [16]:
cgisql.createOrReplaceTempView("cgisql")

In [25]:
mdsql9.createOrReplaceTempView("mdsql9")

In [26]:
mdsql9 = spark.sql("select *, lower(Medication) as lower_Medication from mdsql9")
mdsql9.createOrReplaceTempView("mdsql9")

In [23]:
cgijoin = spark.sql("select mdcleaned.Medication, cgisql.* from cgisql inner join mdcleaned on cgisql.siteId = mdcleaned.siteID AND cgisql.BackgroundID = mdcleaned.BackgroundID AND cgisql.ednum = mdcleaned.ednum")


In [25]:
cgijoin.createOrReplaceTempView("cgijoin")

In [26]:
cgigroup = spark.sql("select Medication,Severity,avg(Improvement) AS meanimp from cgijoin group by Medication,Severity")
cgigroup.createOrReplaceTempView("cgigroup")

In [28]:
severity1 = spark.sql("select * from cgigroup where Severity='1'")

In [42]:
severity.createOrReplaceTempView("severity")


NameError: name 'severity' is not defined

In [40]:
describe = spark.sql("select Severity, avg(meanimp) as mean_imp,max(meanimp) as max_imp, min(meanimp) as min_imp from cgigroup group by Severity")
describe.createOrReplaceTempView("describe")


In [ ]:
max = spark.sql("select Severity, max(meanimp) as max_imp from cgigroup group by Severity")

In [46]:
medications = spark.sql("select * , (case when (Severity='7' and meanimp=7.0 ) then 1 else 0 end) as max_flag from cgigroup")
medications.createOrReplaceTempView("medications")
medications = spark.sql("select *  from medications where max_flag=1")
medications.createOrReplaceTempView("medications")


In [53]:
medications = spark.sql("select * , (case when (Severity='7' and meanimp=7.0 ) or (Severity='6' and meanimp=6 ) or (Severity='5' and meanimp=7.0) or (Severity='4' and meanimp=4.3076923076923075) or (Severity='3' and meanimp=4.0) or (Severity='2' and meanimp=4.0) or  (Severity='1' and meanimp=4.0) then 1 else 0 end) as max_flag from cgigroup")
medications.createOrReplaceTempView("medications")
medications = spark.sql("select *  from medications where max_flag=1 order by Severity")
medications.createOrReplaceTempView("medications")


In [54]:
medications.take(100)

[Row(Medication='librax', Severity='1', meanimp=4.0, max_flag=1),
 Row(Medication='periactin', Severity='2', meanimp=4.0, max_flag=1),
 Row(Medication='cyproheptadine', Severity='2', meanimp=4.0, max_flag=1),
 Row(Medication='buprenorphine', Severity='2', meanimp=4.0, max_flag=1),
 Row(Medication='cytomel', Severity='3', meanimp=4.0, max_flag=1),
 Row(Medication='galantamine', Severity='3', meanimp=4.0, max_flag=1),
 Row(Medication='acamprosate', Severity='4', meanimp=4.3076923076923075, max_flag=1),
 Row(Medication='naltrexone', Severity='5', meanimp=7.0, max_flag=1),
 Row(Medication='focalin', Severity='6', meanimp=6.0, max_flag=1),
 Row(Medication='invega', Severity='7', meanimp=7.0, max_flag=1),
 Row(Medication='ondansetron', Severity='7', meanimp=7.0, max_flag=1),
 Row(Medication='campral', Severity='7', meanimp=7.0, max_flag=1),
 Row(Medication='zyprexa', Severity='7', meanimp=7.0, max_flag=1),
 Row(Medication='dilaudid', Severity='7', meanimp=7.0, max_flag=1),
 Row(Medication='p